<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#⭐-STAR-⭐" data-toc-modified-id="⭐-STAR-⭐-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>⭐ STAR ⭐</a></span></li><li><span><a href="#📦-Imports" data-toc-modified-id="📦-Imports-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>📦 Imports</a></span></li><li><span><a href="#💾-Read-Data" data-toc-modified-id="💾-Read-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>💾 Read Data</a></span></li><li><span><a href="#🧼-Cleaning" data-toc-modified-id="🧼-Cleaning-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>🧼 Cleaning</a></span></li><li><span><a href="#🔨-Workflow" data-toc-modified-id="🔨-Workflow-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>🔨 Workflow</a></span><ul class="toc-item"><li><span><a href="#Creating-Summary-for-Posting" data-toc-modified-id="Creating-Summary-for-Posting-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Creating Summary for Posting</a></span></li><li><span><a href="#Summarizing-the-Summary" data-toc-modified-id="Summarizing-the-Summary-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Summarizing the Summary</a></span></li></ul></li><li><span><a href="#👨‍💻-So-Far..." data-toc-modified-id="👨‍💻-So-Far...-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>👨‍💻 So Far...</a></span></li><li><span><a href="#✔-Checkpoint---4.19.22" data-toc-modified-id="✔-Checkpoint---4.19.22-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>✔ Checkpoint - 4.19.22</a></span></li><li><span><a href="#📄-Final-Script" data-toc-modified-id="📄-Final-Script-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>📄 Final Script</a></span></li></ul></div>

# Intro

# ⭐ STAR ⭐

🔎 **Situation** 🔍

---

* NA uses DRR summary reports to post revenue in LS
* I use report packet to review for DRR

---

⚠️ **Threat** ⚠️

---

* **Pro:** clearly provide total sum amounts for audit to post directly

* **Cons:**
    * "Food" = "Food" + "Other"
    * Discounts are ambiguous
        * FBC vs. Assoc vs. others
    * *LOTS* of unused rows/labels create visual confusion
    * Not always clear to which XAC to post revenue/tax/tips
    * Unusual/new/unused categories are not always accounted for

All issues result in manual review of all transactions during the Daily reporting process.

---
    

⚒️ **Action** ⚒️

---

**Create workflow to:**

* *Read existing DD reports*


* *Clean the report to isolate relevant details*


* *Report any irregularities:*
    * Any discounts or charges?
    * Any new/non-standard MOPs?
    * Report exact amounts to post to each LS XAC
        * See below

* Python Logging Documentation:
    * References:
        * [Real Python](https://realpython.com/python-logging/)
        * [Python Docs](https://docs.python.org/3/howto/logging.html#logging-basic-tutorial)
---

---

**End Goal:**

---

> **Post these amounts in LightSpeed:**

<!-- | |Dept | Rain 903 | IRD  | CC | Bar| 
| - | - | - | - | - | - |
|**Sub-Dept** | |  |  |  |
|Food | 1.00 | 1.00 | 1.00 | 1.00 |
|Liquor | 2.00 | 2.00 | 2.00 | 2.00 
|Tax | 3.00 | 3.00 | 3.00 | 3.00 
|Gratuity | 4.00 | 4.00 | 4.00 | 4.00  -->

|Dept| Food | Liquor | Tax  | Gratuity | 
| -  | -    | -      | -    | -        |
| DR | 1.00 | 1.00   | 1.00 | 1.00     |
| RS | 4.00 | 4.00   | 4.00 | 4.00     |
| CC | 1.00 | 1.00   | 1.00 | 1.00     |
| LB | 4.00 | 4.00   | 4.00 | 4.00     |

---

 >**Investigate these amounts:**
 
 | Item            | Amount |
 | -               | - |
 | Discount        | 1.00 |
 | Charge          | 1.00 |
 | $2 Tip - Coupon | 1.00 |

---


📋 **Result** 📋

# 📦 Imports

---

I will **use standard data reading/wrangling/EDA packages,** such as Numpy, Pandas, Matplotlib, and Seaborn, to prepare and review my data prior to generating the final report.

Additionally, I will **import my own personal EDA module,** containing a selection of hand-written functions to make the EDA process a bit easier.

Finally, I will **create two particular variables to help determine the source data:**
1. A Boolean variable controlling whether to use the reference files included in this repo (versus live/new reports).
2. A filepath as a string datatype to import the data.

---

In [1]:
## Importing Packages

import numpy as np
import pandas as pd
import bmcds.eda as eda
from functions.script import *

import matplotlib.pyplot as plt
import seaborn as sns

import logging

# from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)
%matplotlib inline

In [2]:
## Importing and instantiating logger for future script useage.

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s', 
                    datefmt='%d-%b-%y %H:%M:%S',filename='logs.log',filemode='a')

## Adapted from http://docs.python.org/howto/logging-cookbook.html#logging-cookbook

## Create logger and set level
logger = logging.getLogger('Report_Logs')
logger.setLevel(logging.INFO)

## Create log file
fh = logging.FileHandler('logs.log')
fh.setLevel(logging.INFO)

## Setting formatting
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

logger.addHandler(fh)

# 💾 Read Data

In [3]:
## Specify file path

file_path = './data/02_25_2022_pc.xls'

In [4]:
## Read all columns from file (incl. empty columns)

df = pd.read_excel(file_path, skiprows = 2)
df

,Bar,Unnamed: 1,Dining Room,Unnamed: 3,Room Service,Unnamed: 5,Starbucks,Unnamed: 7,Page Total,Unnamed: 9
0,Food,742.00,NaN,273.00,NaN,62.00,NaN,247.65,NaN,1324.65
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Beverage,1215.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,1215.00
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Other,0.00,NaN,0.00,NaN,20.00,NaN,16.25,NaN,36.25
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Sales Subtotal,1957.00,NaN,273.00,NaN,82.00,NaN,263.90,NaN,2575.90
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Discount,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 🧼 Cleaning

---

I imported the data into a dataframe and immediately see that I will need to fix the alignment of my column names.

Additionally, there are several "Unnamed" columns (representing empty, unlabeled columns generated as part of the report). Knowing those columns are empty, I will drop those as well.

---

In [5]:
## Shifting label names one to the right and drop extra column

df.columns = ['Label', *df.columns[:-1]]
df

,Label,Bar,Unnamed: 1,Dining Room,Unnamed: 3,Room Service,Unnamed: 5,Starbucks,Unnamed: 7,Page Total
0,Food,742.00,NaN,273.00,NaN,62.00,NaN,247.65,NaN,1324.65
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Beverage,1215.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,1215.00
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Other,0.00,NaN,0.00,NaN,20.00,NaN,16.25,NaN,36.25
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Sales Subtotal,1957.00,NaN,273.00,NaN,82.00,NaN,263.90,NaN,2575.90
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Discount,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
## Creating list of columns to keep/drop

col_labels = ['Label', 'Bar', 'Dining Room', 'Room Service', 'Starbucks']
drop_cols = []

for col in df.columns:
    if col not in col_labels:
        drop_cols.append(col)
        
drop_cols

['Unnamed: 1', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7', 'Page Total']

In [7]:
## Dropping columns not representing an outlet

df = df.drop(columns = drop_cols)
df

,Label,Bar,Dining Room,Room Service,Starbucks
0,Food,742.00,273.00,62.00,247.65
1,NaN,NaN,NaN,NaN,NaN
2,Beverage,1215.00,0.00,0.00,0.00
3,NaN,NaN,NaN,NaN,NaN
4,Other,0.00,0.00,20.00,16.25
5,NaN,NaN,NaN,NaN,NaN
6,Sales Subtotal,1957.00,273.00,82.00,263.90
7,NaN,NaN,NaN,NaN,NaN
8,Discount,0.00,0.00,0.00,0.00
9,NaN,NaN,NaN,NaN,NaN


---

Now that I re-labeled the columns to match the data, I see there are blank rows between each row, similar to the blank columns experienced above.

Knowing that the reports do not have any rows that are either missing values or completely blank, I will drop rows with any blank values.

---

In [8]:
## How many rows have missing values, and how many do they contain?

df.isna().sum(axis=1).value_counts()
# df.isna().sum(axis=0)

5    64
0    61
4     1
dtype: int64

In [9]:
## Displaying only rows without any missing values

df[df.isna().sum(axis=1) == 0]

,Label,Bar,Dining Room,Room Service,Starbucks
0,Food,742.00,273.00,62.00,247.65
2,Beverage,1215.00,0.00,0.00,0.00
4,Other,0.00,0.00,20.00,16.25
6,Sales Subtotal,1957.00,273.00,82.00,263.90
8,Discount,0.00,0.00,0.00,0.00
10,Charge,0.00,0.00,0.00,0.00
12,Total before tax,1957.00,273.00,82.00,263.90
14,MD Food 6%,44.52,16.38,3.72,16.04
16,MD Liq 9%,109.35,0.00,0.00,0.00
18,None,0.00,0.00,0.00,0.00


In [10]:
## Displaying only rows missing any values

# df[df.isna().sum(axis=1) > 0]
df[df.isna().sum(axis=1) > 0].value_counts(dropna=False)

Label                  Bar  Dining Room  Room Service  Starbucks
NaN                    NaN  NaN          NaN           NaN          64
*** End of Report ***  NaN  NaN          NaN           NaN           1
dtype: int64

---

The simple filtering above confirms my assumption that rows missing any number of missing values do not contain any relevant information. In the normal printout, these rows would be blank spaces between lines.

Additionally, I noticed there are a few rows that are "blank;" they contain some sort of data, but they do not provide meaningful information.

I will filter out rows with missing values and blank rows.

---

In [11]:
## Checking size of dataframe prior to dropping rows
df.shape

(126, 5)

In [12]:
## Checking for blank rows
df['Label'].value_counts()

                               5
None                           2
Unadjusted receipts            2
Food                           1
Net tips                       1
Paper Gift Certificate         1
LevelUp                        1
S & P                          1
20% Discount                   1
Bad Payment                    1
External Payment               1
Total Paid Outs                1
Server bank                    1
Total tips                     1
Server credit card fees        1
Other receipts turned in       1
Net Cash                       1
$2 Tip - Coupon                1
Server Drops                   1
Turned in                      1
Restaurant credit card fees    1
Estimated deposit              1
Beginning NRS Total            1
Total Receipts                 1
Discounts                      1
Paid Ins                       1
Gross Total                    1
NRS Total                      1
PMS A&G                        1
MasterCard                     1
Visa      

In [13]:
## Reviewing rows containing a blank space for a label
df[df.loc[:, "Label"].isin([' '])]

,Label,Bar,Dining Room,Room Service,Starbucks
28,,,,,
40,,,,,
50,,,,,
82,,,,,
108,,,,,


In [14]:
## Filter out those rows with missing/blank values

df_cleaned = df[df.isna().sum(axis=1) == 0]
df_cleaned = df_cleaned[~df_cleaned.loc[:, "Label"].isin([' '])]
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned

,Label,Bar,Dining Room,Room Service,Starbucks
0,Food,742.00,273.00,62.00,247.65
1,Beverage,1215.00,0.00,0.00,0.00
2,Other,0.00,0.00,20.00,16.25
3,Sales Subtotal,1957.00,273.00,82.00,263.90
4,Discount,0.00,0.00,0.00,0.00
5,Charge,0.00,0.00,0.00,0.00
6,Total before tax,1957.00,273.00,82.00,263.90
7,MD Food 6%,44.52,16.38,3.72,16.04
8,MD Liq 9%,109.35,0.00,0.00,0.00
9,None,0.00,0.00,0.00,0.00


In [15]:
## Checking number of rows post-drop

df_cleaned.shape

(56, 5)

In [16]:
## Inspecting datatype and remaining data
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Label         56 non-null     object
 1   Bar           56 non-null     object
 2   Dining Room   56 non-null     object
 3   Room Service  56 non-null     object
 4   Starbucks     56 non-null     object
dtypes: object(5)
memory usage: 2.3+ KB


---

**Review Current Status**

At this point, I completed the following:

- [X] Imported specified report data
- [X] Removed blank columns and rows
- [X] Removed any rows missing values

At this point, I condensed the report and removed any extra spacing.

**Now I need to decide which rows contain relevant data to keep and which rows contain extra data that we don't use.**

---

**Keeping vs. Dropping**

My original intention for this notebook/eventual script was to create a very minimalistic summary of the relevant day-to-day data, cutting out the extra stuff that we ignore.
- This would provide me with the data specifically used during our night audit processes and remove any confusing extra details.

However, I realized that approach would cause problems if/when we introduce new payment methods or would have any rare situations where relevant data would come through to the reports. 
- Missing such data could result in problems creating the daily reports or worse - obscure potential problems/irregularities.

---

**Final Decision:**

I will focus my code on keeping all row labels while maintaining a focus on the most important day-to-day details.
- To address the concern about irregularities, I will create warnings to highlight such data.

---

# 🔨 Workflow

---

To create my ideal report, **I will focus on specific rows that I use most often for my reporting.** 

I will split the remaining rows into two groups:
1. Regularly-occurring (but irrelevant) data
    - This includes subtotals and most of the rows at the end
    - These rows are irrelevant for the day-to-day reports


2. Infrequent but highly valuable data
    - This group would also act as a catch-all for any new rows, such as when we create a new payment method.

**I will sub-divide the dataframe into the three new dataframes, one for each group.**

- Both the "regular" data and "infrequent-but-valuable" dataframes will be included on the first sheet of the final Excel workbook
- The extra dataframe will be saved to the second page.

---

In [17]:
## Create new DF

df_all_cols = df_cleaned.copy().T
df_all_cols

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
Label,Food,Beverage,Other,Sales Subtotal,Discount,Charge,Total before tax,MD Food 6%,MD Liq 9%,None,...,Server Drops,Turned in,Restaurant credit card fees,Estimated deposit,Beginning NRS Total,Total Receipts,Discounts,Paid Ins,Gross Total,NRS Total
Bar,742.00,1215.00,0.00,1957.00,0.00,0.00,1957.00,44.52,109.35,0.00,...,0.00,2110.87,0.00,2110.87,0.00,2449.85,0.00,0.00,2449.85,2449.85
Dining Room,273.00,0.00,0.00,273.00,0.00,0.00,273.00,16.38,0.00,0.00,...,0.00,289.38,0.00,289.38,0.00,342.88,0.00,0.00,342.88,342.88
Room Service,62.00,0.00,20.00,82.00,0.00,0.00,82.00,3.72,0.00,0.00,...,0.00,85.72,0.00,85.72,0.00,113.12,0.00,0.00,113.12,113.12
Starbucks,247.65,0.00,16.25,263.90,0.00,0.00,263.90,16.04,0.00,0.00,...,0.00,279.94,0.00,279.94,0.00,305.94,0.00,0.00,305.94,305.94


In [18]:
## Re-labeling columns based on "Label" row
df_all_cols.columns = df_all_cols.iloc[0]
df_all_cols = df_all_cols.drop(index = 'Label')
df_all_cols

Label,Food,Beverage,Other,Sales Subtotal,Discount,Charge,Total before tax,MD Food 6%,MD Liq 9%,None,...,Server Drops,Turned in,Restaurant credit card fees,Estimated deposit,Beginning NRS Total,Total Receipts,Discounts,Paid Ins,Gross Total,NRS Total
Bar,742.00,1215.00,0.00,1957.00,0.00,0.00,1957.00,44.52,109.35,0.00,...,0.00,2110.87,0.00,2110.87,0.00,2449.85,0.00,0.00,2449.85,2449.85
Dining Room,273.00,0.00,0.00,273.00,0.00,0.00,273.00,16.38,0.00,0.00,...,0.00,289.38,0.00,289.38,0.00,342.88,0.00,0.00,342.88,342.88
Room Service,62.00,0.00,20.00,82.00,0.00,0.00,82.00,3.72,0.00,0.00,...,0.00,85.72,0.00,85.72,0.00,113.12,0.00,0.00,113.12,113.12
Starbucks,247.65,0.00,16.25,263.90,0.00,0.00,263.90,16.04,0.00,0.00,...,0.00,279.94,0.00,279.94,0.00,305.94,0.00,0.00,305.94,305.94


In [19]:
## Specifying daily-use labels

daily_rows = ['Food', 'Beverage', 'Other', 'Discount', 'MD Food 6%',
              'MD Liq 9%', 'Tip collected', "ALL A&G CHRG", 'Room Charge',
              'American Express', 'Cash', 'Discover Card', 'MasterCard', 'Visa']

df_daily = df_all_cols[daily_rows]
df_daily

Label,Food,Beverage,Other,Discount,MD Food 6%,MD Liq 9%,Tip collected,ALL A&G CHRG,Room Charge,American Express,Cash,Discover Card,MasterCard,Visa
Bar,742.00,1215.00,0.00,0.00,44.52,109.35,338.98,31.25,585.90,91.79,240.84,134.73,334.45,1030.89
Dining Room,273.00,0.00,0.00,0.00,16.38,0.00,53.50,31.20,102.20,0.00,0.00,0.00,58.32,151.16
Room Service,62.00,0.00,20.00,0.00,3.72,0.00,27.40,0.00,70.88,0.00,0.00,0.00,0.00,42.24
Starbucks,247.65,0.00,16.25,0.00,16.04,0.00,26.00,0.00,46.80,10.22,35.60,0.00,31.38,181.94


In [20]:
## Converting data to "float"
for col in df_daily.columns:
    df_daily.loc[:,col] = pd.to_numeric(df_daily.loc[:,col], downcast = 'float')
    
df_daily

<ipython-input-20-04263d01f862>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_daily.loc[:,col] = pd.to_numeric(df_daily.loc[:,col], downcast = 'float')


Label,Food,Beverage,Other,Discount,MD Food 6%,MD Liq 9%,Tip collected,ALL A&G CHRG,Room Charge,American Express,Cash,Discover Card,MasterCard,Visa
Bar,742.00,1215.00,0.00,0.00,44.52,109.35,338.98,31.25,585.90,91.79,240.84,134.73,334.45,1030.89
Dining Room,273.00,0.00,0.00,0.00,16.38,0.00,53.50,31.20,102.20,0.00,0.00,0.00,58.32,151.16
Room Service,62.00,0.00,20.00,0.00,3.72,0.00,27.40,0.00,70.88,0.00,0.00,0.00,0.00,42.24
Starbucks,247.65,0.00,16.25,0.00,16.04,0.00,26.00,0.00,46.80,10.22,35.60,0.00,31.38,181.94


## Creating Summary for Posting

---

**My main goal is to create a simple, easy-to-use report that I can roll-out quickly and easily.** Ideally, the summary will be intuitive, clear, and allow for the night auditor to post the charges quickly and easily.

I know that there are only a select few columns that we reference for posting charges, so I will start my summary by creating a small dataframe summarizing the usual charges.

---

In [21]:
## Summarize food charges by outlet
daily_food = df_daily['Food'] + df_daily['Other'] - df_daily['Discount']
daily_food

Bar            742.00
Dining Room    273.00
Room Service    82.00
Starbucks      263.90
dtype: float32

In [22]:
## Summarize tax charges by outlet
daily_tax = df_daily['MD Food 6%']+df_daily['MD Liq 9%']
daily_tax

Bar            153.87
Dining Room     16.38
Room Service     3.72
Starbucks       16.04
dtype: float32

In [23]:
## Summarize all charges by outlet
summary = pd.concat([daily_food, df_daily['Beverage'], daily_tax, 
           df_daily['Tip collected']], axis=1)
summary

,0,Beverage,1,Tip collected
Bar,742.00,1215.00,153.87,338.98
Dining Room,273.00,0.00,16.38,53.50
Room Service,82.00,0.00,3.72,27.40
Starbucks,263.90,0.00,16.04,26.00


In [24]:
## Creating new labels

col_labels = {0: "Food", 1: "Tax", 'Tip collected': 'Gratuity'}

row_labels = {'Bar': "Lobby Bar", "Dining Room": 'Rain 903', 
              'Room Service': 'In-Room Dining', 'Starbucks': 'Coffee Corner'}

col_labels, row_labels

({0: 'Food', 1: 'Tax', 'Tip collected': 'Gratuity'},
 {'Bar': 'Lobby Bar',
  'Dining Room': 'Rain 903',
  'Room Service': 'In-Room Dining',
  'Starbucks': 'Coffee Corner'})

In [25]:
## Renaming columns
summary = summary.rename(columns = col_labels, index = row_labels)
summary

,Food,Beverage,Tax,Gratuity
Lobby Bar,742.00,1215.00,153.87,338.98
Rain 903,273.00,0.00,16.38,53.50
In-Room Dining,82.00,0.00,3.72,27.40
Coffee Corner,263.90,0.00,16.04,26.00


In [26]:
## Rounding values - preventing long numbers
summary = summary.applymap(lambda x: round(x, 2))
summary

,Food,Beverage,Tax,Gratuity
Lobby Bar,742.00,1215.00,153.87,338.98
Rain 903,273.00,0.00,16.38,53.50
In-Room Dining,82.00,0.00,3.72,27.40
Coffee Corner,263.90,0.00,16.04,26.00


In [27]:
## Saving results
# summary.to_excel('Test_Summary_Workbook.xlsx')

## Summarizing the Summary

---

First summary working - need to expand

* Repeat process per MOP
    * A&G
    * RC
    * CA
    * AX
    * DI
    * VI/MC
    
* Save results to different sheets?

---

# 👨‍💻 So Far...

---

**√: MVP**

- [x] Created function to clean, organize data from summary reports
 - [x] Tested results by looping through list of files


**TD: MVP**

 - [ ] Add additional column identifying MOP
 - [ ] Create multi-index w/ MOP, PC
 - [ ] Concat DFs
 - [ ] Save results to XL
 - [ ] Convert to script
 
 
**TD: AAB**

 - [ ] Create filters to include extra details
     * *Ex: Discounts, charges, any new MOPs, payments other than listed MOPs*

---

# ✔ Checkpoint - 4.19.22

---

**Current Status:**

* Read Digital Dining report data *only for Excel files*
    * AAB: read DBF files directly
    * AAB: review sales type for discounts, subtract from proper sales type
        * Currently subtracted from food generally
* Clean the data to generate summaries about food/bev/tax/gratuity amounts
    * MVP: confirm process works for all MOPs
* Log warnings about any "Duplicate Check" or direct A&G postings

---

**Next Steps:**

1. Test on most recent files
2. Artificially generate errors (then cancel before NA)
3. Condense into script to run on server
4. Test script
5. Schedule script to run daily

---

# 📄 Final Script

Two parts:

1. Run through workflow above using the overall summary data
2. Loop through a list of the usual MOPs and create a multi-indexed DataFrame containing the results for each outlet.

In [51]:
list_mops = ['ag', 'rc', 'ax', 'ca', 'di', 'vimc']

for mop in list_mops:
    
    workflow(data_filepath =f'./data/02_25_2022_mop_det_{mop}.xls', 
             receipts_filepath ='./data/april_2022_rc_receipts.xls',
             log_filepath = 'test_workflow_log.log',
             mop_name = mop)